# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    ROOT,
)
from llm_manager import GraphPrompterHFClassifier

In [2]:
EPOCHS = 4
BATCH_SIZE_KGE = 128000
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[610] },
  target={
    node_id=[9742],
    x=[9742, 20],
  },
  (source, edge, target)={ edge_index=[2, 100836] },
  (target, rev_edge, source)={ edge_index=[2, 100836] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_graph_prompter_hf = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    force_recompute=False,
)

loading pretrained model
Device: 'cpu'


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
INPUT_EMBEDS_REPLACE_ROOT = f"{ROOT}/llm/graph_prompter_hf"

In [7]:
graph_prompter_hf_bert_classifier = GraphPrompterHFClassifier(
    kg_manager,
    graph_representation_generator_graph_prompter_hf.get_embeddings,
    root_path=INPUT_EMBEDS_REPLACE_ROOT,
)

Some weights of GraphPrompterHFBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GraphPrompterHFBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized because the shapes did not match:
- bert.embeddings.token_type_embeddings.weight: found shape torch.Size([2, 128]) in the checkpoint and torch.Size([5, 128]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


5
5
5
device cuda


In [8]:
dataset_embedding = kg_manager.generate_graph_prompter_hf_embedding_dataset(
    graph_prompter_hf_bert_classifier.tokenizer.sep_token,
    graph_prompter_hf_bert_classifier.tokenizer.pad_token,
    graph_prompter_hf_bert_classifier.tokenize_function,
)

In [9]:
graph_prompter_hf_bert_classifier.train_model_on_data(
    dataset_embedding, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)
graph_representation_generator_graph_prompter_hf.save_model()

  0%|          | 0/380 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.7653, 'grad_norm': 2.4647297859191895, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.11}
{'loss': 0.7592, 'grad_norm': 2.3033061027526855, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.21}
{'loss': 0.7605, 'grad_norm': 2.451728582382202, 'learning_rate': 3e-06, 'epoch': 0.32}
{'loss': 0.7547, 'grad_norm': 2.5171608924865723, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.42}
{'loss': 0.7434, 'grad_norm': 2.2348196506500244, 'learning_rate': 5e-06, 'epoch': 0.53}
{'loss': 0.7274, 'grad_norm': 1.9536980390548706, 'learning_rate': 6e-06, 'epoch': 0.63}
{'loss': 0.7162, 'grad_norm': 1.7177497148513794, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.74}
{'loss': 0.7028, 'grad_norm': 1.2595056295394897, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.84}
{'loss': 0.6911, 'grad_norm': 0.8440528512001038, 'learning_rate': 9e-06, 'epoch': 0.95}


  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.7002013325691223, 'eval_accuracy': 0.5, 'eval_runtime': 28.4068, 'eval_samples_per_second': 709.9, 'eval_steps_per_second': 2.781, 'epoch': 1.0}
{'loss': 0.6817, 'grad_norm': 0.6681752800941467, 'learning_rate': 1e-05, 'epoch': 1.05}
{'loss': 0.6767, 'grad_norm': 0.5098661184310913, 'learning_rate': 1.1000000000000001e-05, 'epoch': 1.16}
{'loss': 0.671, 'grad_norm': 0.49885427951812744, 'learning_rate': 1.2e-05, 'epoch': 1.26}
{'loss': 0.671, 'grad_norm': 0.250542014837265, 'learning_rate': 1.3000000000000001e-05, 'epoch': 1.37}
{'loss': 0.6694, 'grad_norm': 0.33521291613578796, 'learning_rate': 1.4000000000000001e-05, 'epoch': 1.47}
{'loss': 0.6699, 'grad_norm': 0.22820617258548737, 'learning_rate': 1.5e-05, 'epoch': 1.58}
{'loss': 0.6708, 'grad_norm': 0.2357388585805893, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.68}
{'loss': 0.6703, 'grad_norm': 0.2652447819709778, 'learning_rate': 1.7000000000000003e-05, 'epoch': 1.79}
{'loss': 0.661, 'grad_norm': 0.2516632

...

{'eval_loss': 0.719588577747345, 'eval_accuracy': 0.5, 'eval_runtime': 28.2234, 'eval_samples_per_second': 714.514, 'eval_steps_per_second': 2.799, 'epoch': 2.0}
{'loss': 0.6619, 'grad_norm': 0.3280428647994995, 'learning_rate': 2e-05, 'epoch': 2.11}
{'loss': 0.6627, 'grad_norm': 0.49931812286376953, 'learning_rate': 2.1e-05, 'epoch': 2.21}
{'loss': 0.6611, 'grad_norm': 0.37573713064193726, 'learning_rate': 2.2000000000000003e-05, 'epoch': 2.32}
{'loss': 0.6602, 'grad_norm': 0.4987717568874359, 'learning_rate': 2.3000000000000003e-05, 'epoch': 2.42}
{'loss': 0.6486, 'grad_norm': 0.5621449947357178, 'learning_rate': 2.4e-05, 'epoch': 2.53}
{'loss': 0.6594, 'grad_norm': 0.4942534565925598, 'learning_rate': 2.5e-05, 'epoch': 2.63}
{'loss': 0.6487, 'grad_norm': 0.4540857672691345, 'learning_rate': 2.6000000000000002e-05, 'epoch': 2.74}
{'loss': 0.6443, 'grad_norm': 0.7696588039398193, 'learning_rate': 2.7000000000000002e-05, 'epoch': 2.84}
{'loss': 0.6378, 'grad_norm': 0.6840952634811401, 

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.6727188229560852, 'eval_accuracy': 0.5960031736586333, 'eval_runtime': 28.6297, 'eval_samples_per_second': 704.375, 'eval_steps_per_second': 2.759, 'epoch': 3.0}
{'loss': 0.6171, 'grad_norm': 0.7890952229499817, 'learning_rate': 2.9e-05, 'epoch': 3.05}
{'loss': 0.6127, 'grad_norm': 0.7773005962371826, 'learning_rate': 3e-05, 'epoch': 3.16}
{'loss': 0.6015, 'grad_norm': 0.8128824234008789, 'learning_rate': 3.1e-05, 'epoch': 3.26}
{'loss': 0.5942, 'grad_norm': 1.00925874710083, 'learning_rate': 3.2000000000000005e-05, 'epoch': 3.37}
{'loss': 0.5854, 'grad_norm': 0.9878221154212952, 'learning_rate': 3.3e-05, 'epoch': 3.47}
{'loss': 0.5842, 'grad_norm': 1.1933306455612183, 'learning_rate': 3.4000000000000007e-05, 'epoch': 3.58}
{'loss': 0.5715, 'grad_norm': 0.7455223798751831, 'learning_rate': 3.5e-05, 'epoch': 3.68}
{'loss': 0.5699, 'grad_norm': 0.9083303809165955, 'learning_rate': 3.6e-05, 'epoch': 3.79}
{'loss': 0.5674, 'grad_norm': 0.8539912104606628, 'learning_rate': 3

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.591907262802124, 'eval_accuracy': 0.7134781315084796, 'eval_runtime': 28.3458, 'eval_samples_per_second': 711.428, 'eval_steps_per_second': 2.787, 'epoch': 4.0}
{'train_runtime': 298.6113, 'train_samples_per_second': 324.181, 'train_steps_per_second': 1.273, 'train_loss': 0.6598376562720851, 'epoch': 4.0}
